In [1]:
import logging
# from Controller import Controller, Controller_sequence, Controller_pure
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
import argparse
from multiprocessing import Pool, cpu_count, Process
import multiprocessing
# from utils import mod_column, evaluate, init_name_and_log, save_result
from collections import ChainMap
from subprocess import Popen, PIPE
from time import time, sleep
import os
# from Java_service import start_service_pool, stop_service_pool, find_free_port
# import rpyc
import random
import tensorflow as tf

In [2]:
def get_reward(actions):
	global path, num_op_unary, num_op_binary, max_order, num_batch, optimizer, lr, epochs, evaluate, task, dataset, model, alpha, lr_value, RL_model, reg, controller, num_random_sample, lambd, multiprocessing, package, method, origin_result
	X = pd.read_csv(path)
	num_feature = X.shape[1] - 1
	action_per_feature = int(len(actions) / num_feature)
	copies, copies_run, rewards = {}, [], []

	for feature_count in range(num_feature):
		feature_name = X.columns[feature_count]
		feature_actions = actions[feature_count*action_per_feature: \
			(feature_count+1)*action_per_feature]
		copies[feature_count] = []
		if feature_actions[0] == 0:
			continue
		else:
			copy = np.array(X[feature_name].values)		

		for action in feature_actions:
			if action == 0:
				break
			elif action > 0 and action <= num_op_unary:
				action_unary = action - 1
				if action_unary == 0:
					copy = np.squeeze(np.sqrt(abs(copy)))
				elif action_unary == 1:
					scaler = MinMaxScaler()
					copy = np.squeeze(scaler.fit_transform(np.reshape(copy,[-1,1])))
				elif action_unary == 2:
					while (np.any(copy == 0)):
						copy = copy + 1e-5
					copy = np.squeeze(np.log(abs(np.array(copy))))
				elif action_unary == 3:
					while (np.any(copy == 0)):
						copy = copy + 1e-5
					copy = np.squeeze(1 / (np.array(copy))) 
			
			else:
				action_binary = (action-num_op_unary-1) // (num_feature-1)
				rank = np.mod(action-num_op_unary-1, num_feature-1)

				if rank >= feature_count:
					rank += 1
				target_feature_name = X.columns[rank]

				target = np.array(X[target_feature_name].values) 

				if action_binary == 0:
					copy = np.squeeze(copy + target)
				elif action_binary == 1:
					copy = np.squeeze(copy - target)
				elif action_binary == 2:
					copy = np.squeeze(copy * target)
				elif action_binary == 3:
					while (np.any(target == 0)):
						target = target + 1e-5
					copy = np.squeeze(copy / target) 
				elif action_binary == 4:
					copy = np.squeeze(mod_column(copy, X[target_feature_name].values))

			copies[feature_count].append(copy)
		copies_run.append(copy)

	if method == 'train':
		former_result = origin_result
		former_copys = [None]
		for key in sorted(copies.keys()):
			reward, former_result, return_copy = get_reward_per_feature( 
				copies[key], action_per_feature, former_result, former_copys)
			former_copys.append(return_copy)
			rewards += reward
		return rewards
	
	elif method == 'test':
		for i in range(len(copies_run)):
			X.insert(0, 'new%d'%i, copies_run[i])
		if package == 'weka':
			result = get_weka_result(X)
		elif package == 'sklearn':
			y = X[X.columns[-1]]
			del X[X.columns[-1]]
			result = evaluate1(X, y, num_op_unary, num_op_binary, max_order, num_batch, optimizer, lr, epochs, evaluate, task, dataset, model, alpha, lr_value, RL_model, reg, controller, num_random_sample, lambd, multiprocessing, package)
		return result
		

def get_reward_per_feature(copies, count, former_result, former_copys=[None]):
	global path, num_op_unary, num_op_binary, max_order, num_batch, optimizer, lr, epochs, evaluate, task, dataset, model, alpha, lr_value, RL_model, reg, controller, num_random_sample, lambd, multiprocessing, package
	X = pd.read_csv(path)
	if package == 'sklearn':
		y = X[X.columns[-1]]
		del X[X.columns[-1]]

	reward = []
	previous_result = former_result
	for i,former_copy in enumerate(former_copys):
		if not former_copy is None:
			X.insert(0, 'former%d'%i, former_copy)

	for copy in copies:
		X.insert(0, 'new', copy)
		if package == 'weka':
			current_result = get_weka_result(X)
		elif package == 'sklearn':
			current_result = evaluate1(X, y, num_op_unary, num_op_binary, max_order, num_batch, optimizer, lr, epochs, evaluate, task, dataset, model, alpha, lr_value, RL_model, reg, controller, num_random_sample, lambd, multiprocessing, package)

		reward.append(current_result - previous_result)
		previous_result = current_result
		del X['new']

	reward_till_now = len(reward)
	for _ in range(count - reward_till_now):
		reward.append(0)
	if len(copies) == 0:
		return_copy = None
	else:
		return_copy = copies[-1]

	return reward, previous_result, return_copy

def random_run(nnum_random_sample, nmodel, l=None, p=None):
	global num_op_unary, num_op_binary, max_order, num_batch, optimizer, lr, epochs, evaluate, task, dataset, model, alpha, lr_value, RL_model, reg, controller, num_random_sample, lambd, multiprocessing, package, num_process
	samples = []
	for i in range(nnum_random_sample):
		sample = []
		for _ in range(nmodel.num_action):
			sample.append(np.random.randint(nmodel.num_op))
		samples.append(sample)

	if multiprocessing:	
		# if package == 'weka':
		# 	pool = Pool(num_process, initializer=init, initargs=(l, p))
		# elif package == 'sklearn':
		if package == 'sklearn':
			pool = Pool(num_process)    
		res = list(pool.map(get_reward, samples))
		pool.close()
		pool.join()
	else:
		res = []
		for sample in samples:
			res.append(get_reward(sample))

	random_result = max(res)
	random_sample = samples[res.index(random_result)]

	return random_result, random_sample


def train(nmodel, l=None, p=None):
	global path, num_op_unary, num_op_binary, max_order, num_batch, optimizer, lr, epochs, evaluate, task, dataset, model, alpha, lr_value, RL_model, reg, controller, num_random_sample, lambd, multiprocessing, package, infos, method, origin_result, num_process

	X = pd.read_csv(path)
	print(X)
	# if package == 'weka':
	# 	origin_result = get_weka_result(X)
	# elif package == 'sklearn':
	if package == 'sklearn':
		y = X[X.columns[-1]]
		del X[X.columns[-1]]
		print(X.shape)

		origin_result = evaluate1(X, y, num_op_unary, num_op_binary, max_order, num_batch, optimizer, lr, epochs, evaluate, task, dataset, model, alpha, lr_value, RL_model, reg, controller, num_random_sample, lambd, multiprocessing, package)	
	best_result = origin_result
	print(origin_result)

	config = tf.ConfigProto()
	config.gpu_options.allow_growth = True
	with tf.Session(config=config) as sess:
		init_op = tf.group(tf.global_variables_initializer(), 
			tf.local_variables_initializer())
		sess.run(init_op)
		
		nmodel_result = -10000.0 
		train_set, values = [], []
		for epoch_count in range(epochs):
			concat_action = []
			probs_action = sess.run(tf.nn.softmax(nmodel.concat_output))

			for batch_count in range(num_batch):
				batch_action = []
				for i in range(probs_action.shape[0]):
					sample_action = np.random.choice(len(probs_action[i]), p=probs_action[i])
					batch_action.append(sample_action)
				concat_action.append(batch_action)
					
			method = 'train'
			if multiprocessing:
				# if package == 'weka':
				# 	pool = Pool(num_process, initializer=init, initargs=(l, p))  
				# elif package == 'sklearn':
				if package == 'sklearn':
					pool = Pool(num_process)        
				rewards = np.array(pool.map(get_reward, concat_action))
				pool.close()
				pool.join()
			else:
				rewards = []
				for action in concat_action:
					rewards.append(get_reward(action))
				rewards = np.array(rewards)

			method = 'test'
			if multiprocessing:
				# if package == 'weka':
				# 	pool = Pool(num_process, initializer=init, initargs=(l, p))  
				# elif package == 'sklearn':
				if package == 'sklearn':
					pool = Pool(num_process)      
				results = pool.map(get_reword, concat_action)
				pool.close()
				pool.join()
			else:
				results = []
				for action in concat_action:
					results.append(get_reword(action))
			nmodel_result = max(nmodel_result, max(results))


			if RL_model == 'AC':
				target_set = []
				for batch_count in range(num_batch):
					action = concat_action[batch_count]
					for i in range(nmodel.num_action):
						train_tmp = list(np.zeros(nmodel.num_action, dtype=int))
						target_tmp = list(np.zeros(nmodel.num_action, dtype=int))
						
						train_tmp[0:i] = list(action[0:i])
						target_tmp[0:i+1] = list(action[0:i+1])

						train_set.append(train_tmp)
						target_set.append(target_tmp)

				state = np.reshape(train_set, [-1,nmodel.num_action])
				next_state = np.reshape(target_set, [-1,nmodel.num_action])

				value = nmodel.predict_value(next_state) * alpha + rewards.flatten()
				values += list(value)
				nmodel.update_value(state, values)

				rewards_predict = nmodel.predict_value(next_state) * alpha - \
					nmodel.predict_value(state[-np.shape(next_state)[0]:]) + rewards.flatten()
				rewards = np.reshape(rewards_predict, [num_batch,-1])

			
			elif RL_model == 'PG':
				for i in range(nmodel.num_action):
					base = rewards[:,i:]
					rewards_order = np.zeros_like(rewards[:,i])
					for j in range(base.shape[1]):
						order = j + 1
						base_order = base[:,0:order]
						alphas = []
						for o in range(order):
							alphas.append(pow(alpha, o))
						base_order = np.sum(base_order*alphas, axis=1)
						base_order = base_order * np.power(lambd, j) 
						rewards_order = rewards_order.astype(float) 
						rewards_order += base_order.astype(float) 
					rewards[:,i] = (1-lambd) * rewards_order
				

			feed_dict = {nmodel.concat_action: np.reshape(concat_action, [num_batch,-1]), \
				nmodel.rewards: np.reshape(rewards,[num_batch,-1])}
			loss_epoch = nmodel.update_policy(feed_dict, sess)


			method = 'test'
			probs_action = sess.run(tf.nn.softmax(nmodel.concat_output))
			best_action = probs_action.argmax(axis=1)
			nmodel_result = max(nmodel_result, get_reword(best_action))

			random_result, random_sample = random_run(num_random_sample, nmodel, l, p)

			best_result = max(best_result, nmodel_result)

			print('Epoch %d: loss = %.4f, origin_result = %.4f, lr = %.3f, \n model_result = %.4f, best_action = %s, \n best_result = %.4f, random_result = %.4f, random_sample = %s' 
				% (epoch_count+1, loss_epoch, origin_result, lr, nmodel_result, str(best_action), best_result, random_result, str(random_sample)))
			logging.info('Epoch %d: loss = %.4f, origin_result = %.4f, lr = %.3f, \n model_result = %.4f, best_action = %s, \n best_result = %.4f, random_result = %.4f, random_sample = %s' 
				% (epoch_count+1, loss_epoch, origin_result, lr, nmodel_result, str(best_action), best_result, random_result, str(random_sample)))

			info = [epoch_count, loss_epoch, origin_result, nmodel_result, random_result]
			infos.append(info)
		

In [3]:
def main(pnum_op_unary=4, pnum_op_binary=5, pmax_order=5, pnum_batch=32, poptimizer='adam', 
         plr=0.01, pepochs=100, pevaluate='r2', ptask='regression', pdataset='airfoil', 
         pmodel='RF', palpha=0.99, plr_value=1e-3, pRL_model='PG', preg=1e-5, 
         pcontroller='rnn', pnum_random_sample=5, plambd=0.4, pmultiprocessing=True, ppackage='sklearn'):
	global num_process, infos
	global path, num_op_unary, num_op_binary, max_order, num_batch, optimizer, lr, epochs, evaluate, task, dataset, model, alpha, lr_value, RL_model, reg, controller, num_random_sample, lambd, multiprocessing, package
	num_op_unary = pnum_op_unary
	num_op_binary = pnum_op_binary
	max_order = pmax_order
	num_batch = pnum_batch
	optimizer = poptimizer
	lr = plr
	epochs = pepochs
	evaluate = pevaluate
	task = ptask
	dataset = pdataset
	model = pmodel
	alpha = palpha
	lr_value = plr_value
	RL_model = pRL_model
	reg = preg
	controller = pcontroller
	num_random_sample = pnum_random_sample
	lambd = plambd
	multiprocessing = pmultiprocessing
	package = ppackage

# if __name__ == '__main__':
	# args = parse_args()
	origin_result, method, name = None, None, None
	num_process, infos = 64, []
	# num_weka_process = num_process
	name = init_name_and_log(num_op_unary, num_op_binary, max_order, num_batch, optimizer, lr, epochs, evaluate, task, dataset, model, alpha, lr_value, RL_model, reg, controller, num_random_sample, lambd, multiprocessing, package)
	print(name)


	path = '/content/' + dataset + '.csv'
	num_feature = pd.read_csv(path).shape[1] - 1
	if controller == 'rnn':
		controller = Controller(num_op_unary, num_op_binary, max_order, num_batch, optimizer, 
         lr, epochs, evaluate, task, dataset, 
         model, alpha, lr_value, RL_model, reg, 
         controller, num_random_sample, lambd, multiprocessing, package, num_feature)
	# elif controller == 'pure':
	# 	controller = Controller_pure(args, num_feature)
	controller.build_graph()

	# if package == 'weka':
	# 	train(controller, lock, ports)
	# elif package == 'sklearn':
	# 	train(controller)
	train(controller)

	save_result(infos, name)
	
	# if args.package == 'weka':
	# 	stop_service_pool(pids)
	# 	for port in conns.keys():
	# 		conns[port].close()
	print("end")

In [4]:
global path, num_op_unary, num_op_binary, max_order, num_batch, optimizer, lr, epochs, evaluate, task, dataset, model, alpha, lr_value, RL_model, reg, controller, num_random_sample, lambd, multiprocessing, package, method, origin_result, num_process, infos

# need to implement 1-rae

In [5]:
# BMI
main(5, 4, 5, 32, 'adam', 0.01, 10, 'r2', 'regression', 'BMI', 'RF', 0.99, 1e-3, 'PG', 1e-5, 'rnn', 5, 0.4, True, 'sklearn')

NameError: name 'init_name_and_log' is not defined

In [ ]:
# airfoil

main(5, 4, 5, 32, 'adam', 0.01, 10, 'r2', 'regression', 'airfoil', 'RF', 0.99, 1e-3, 'PG', 1e-5, 'rnn', 5, 0.4, True, 'sklearn')